In [ ]:
pip install sentence-transformers

In [ ]:
pip install mysql-connector-python

In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("shibing624/text2vec-base-chinese")

sentence1_embeddings = model.encode(["go to the curch"])
# sentence2_embeddings = model.encode(["你是誰"])
# sentence2_embeddings = model.encode(["你過得如何"])
sentence2_embeddings = model.encode(["come on"])

similarity = util.pytorch_cos_sim(sentence1_embeddings, sentence2_embeddings)
print(similarity)

In [ ]:
from http.server import BaseHTTPRequestHandler, HTTPServer
from sentence_transformers import SentenceTransformer, util
import mysql.connector
import json

def connect_to_database():
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="3659215fF",
        database="handle_together"
    )
    return connection

def send_json_response(self, status_code, data):
    # Convert status_code to integer
    status_code = int(status_code)

    # Set up the HTTP response
    self.send_response(status_code)
    self.send_header("Content-type", "application/json")
    self.end_headers()

    # Convert the data to JSON
    json_response = json.dumps(data)

    # Send the JSON response
    self.wfile.write(json_response.encode('utf-8'))

class WebServerHandler(BaseHTTPRequestHandler):

    def do_POST(self):       
        # Read the content length from the request headers
        content_length = int(self.headers['Content-Length'])

        # Read the JSON data from the request body
        post_data = self.rfile.read(content_length)
        json_data_list = json.loads(post_data.decode('utf-8'))

        if json_data_list:
            json_data = json_data_list[0]
            # Now json_data contains the parameters sent in the JSON format
            tag_array = json_data.get('tag_array', [])
            subject_array = json_data.get('subject_array', [])
            object_array = json_data.get('object_array', [])
            input_sentence = json_data.get('input_sentence', '')       
            public_ind = json_data.get('public_ind', '')
            command_ind = json_data.get('command_ind', '')

            print()
        # Load the Sentence Transformer model
        model = SentenceTransformer("shibing624/text2vec-base-chinese")

        # Fetch distinct subject codes from intent_type table
        connection = connect_to_database()
        cursor = connection.cursor()

        # Use parameterized query to avoid SQL injection
        # query = "SELECT tag, content FROM intent_training_data WHERE tag IN ({}) AND subject IN ({})".format(','.join(['%s' for _ in tag_array]), ','.join(['%s' for _ in subject_array]))
        if tag_array and any(tag_array):
            tag_condition = "tag IN ({})".format(','.join(['%s' for _ in tag_array]))
        else:
            tag_condition = "1=1"  # Always true if tag_array is empty or None
            
        if command_ind: 
            tag_condition2 = "AND COMMAND_IND = '"+command_ind+"'"
        else:
            tag_condition2 = ""  # Always true if tag_array is empty or None
        
        query = ("SELECT tag, content FROM intent_training_data WHERE {} AND subject IN ({}) AND object IN ({}) AND PUBLIC_IND = '"+public_ind+"'"+tag_condition2).format(tag_condition, ','.join(['%s' for _ in subject_array]) ,','.join(['%s' for _ in object_array]))
        params = tag_array + subject_array + object_array
        cursor.execute(query, params)
        rows = cursor.fetchall()
        candidate_responses_tags = [row[0] for row in rows]
        candidate_responses = [row[1] for row in rows]

        cursor.close()
        connection.close()

        # Encode input sentence and candidate responses
        input_embedding = model.encode([input_sentence])
        response_embeddings = model.encode(candidate_responses)

        # Calculate cosine similarity between input and responses
        similarities = util.pytorch_cos_sim(input_embedding, response_embeddings)[0]
        
        similarity = util.pytorch_cos_sim(input_embedding, response_embeddings)

        # Get the index of the most similar response
        most_similar_index = similarities.argmax().item()

        # Print the most similar response
        print("Input sentence:", input_sentence)
        print("Most tag:", candidate_responses_tags[most_similar_index])
        print("Most similar response:", candidate_responses[most_similar_index])
        print("similarity:",similarity)
        result = {
            "input_sentence": input_sentence,
            "tag": candidate_responses_tags[most_similar_index],
            "content": candidate_responses[most_similar_index],
            "similarity" : round(similarity.item(),5)
        }
        send_json_response(self, '200', result)

if __name__ == '__main__':
    try:
        port = 8000
        server = HTTPServer(('localhost', port), WebServerHandler)
        server.max_body_length = 10 * 1024 * 1024  # set to 10 MB

        print(f'Server running on port {port}')
        
        server.serve_forever()
    except KeyboardInterrupt:
        print('^C received, shutting down server')
        server.socket.close()


In [ ]:
import random
from http.server import BaseHTTPRequestHandler, HTTPServer
from sentence_transformers import SentenceTransformer, util
import mysql.connector
import json
import torch

def connect_to_database():
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="3659215fF",
        database="handle_together"
    )
    return connection

def send_json_response(self, status_code, data):
    # Convert status_code to integer
    status_code = int(status_code)

    # Set up the HTTP response
    self.send_response(status_code)
    self.send_header("Content-type", "application/json")
    self.end_headers()

    # Convert the data to JSON
    json_response = json.dumps(data)

    # Send the JSON response
    self.wfile.write(json_response.encode('utf-8'))

class WebServerHandler(BaseHTTPRequestHandler):

    def do_POST(self):       
        # Read the content length from the request headers
        content_length = int(self.headers['Content-Length'])

        # Read the JSON data from the request body
        post_data = self.rfile.read(content_length)
        json_data_list = json.loads(post_data.decode('utf-8'))

        if json_data_list:
            json_data = json_data_list[0]
            # Now json_data contains the parameters sent in the JSON format
            tag_array = json_data.get('tag_array', [])
            subject_array = json_data.get('subject_array', [])
            object_array = json_data.get('object_array', [])
            input_sentence = json_data.get('input_sentence', '')       
            public_ind = json_data.get('public_ind', '')
            command_ind = json_data.get('command_ind', '')

        # Load the Sentence Transformer model
        model = SentenceTransformer("shibing624/text2vec-base-chinese")

        # Fetch distinct subject codes from intent_type table
        connection = connect_to_database()
        cursor = connection.cursor()

        # Use parameterized query to avoid SQL injection
        # query = "SELECT tag, content FROM intent_training_data WHERE tag IN ({}) AND subject IN ({})".format(','.join(['%s' for _ in tag_array]), ','.join(['%s' for _ in subject_array]))
        if tag_array and any(tag_array):
            tag_condition = "tag IN ({})".format(','.join(['%s' for _ in tag_array]))
        else:
            tag_condition = "1=1"  # Always true if tag_array is empty or None
            
        if command_ind: 
            tag_condition2 = "AND COMMAND_IND = '"+command_ind+"'"
        else:
            tag_condition2 = "" 
        query = ("SELECT tag, content FROM intent_training_data WHERE {} AND subject IN ({}) AND object IN ({}) AND PUBLIC_IND = '"+public_ind+"'"+tag_condition2).format(tag_condition, ','.join(['%s' for _ in subject_array]) ,','.join(['%s' for _ in object_array]))
        params = tag_array + subject_array + object_array
        cursor.execute(query, params)
        rows = cursor.fetchall()
        candidate_responses_tags = [row[0] for row in rows]
        candidate_responses = [row[1] for row in rows]
        # for row in rows:
        #     print(row)
        # print("SQL Query:", query)
        # print("Parameters:", params)
        cursor.close()
        connection.close()

        # Encode input sentence and candidate responses
        input_embedding = model.encode([input_sentence])
        response_embedding = model.encode([candidate_responses])

        response_embeddings = model.encode(candidate_responses)

        # Calculate cosine similarity between input and responses
        similarities = util.pytorch_cos_sim(input_embedding, response_embeddings)[0]
        
        most_similar_index = similarities.argmax().item()
        top_n_indices = torch.argsort(similarities, descending=True)[:5]

        # Randomly select 1 response from the top 5
        selected_index = random.choice(top_n_indices)
        selected_response_tag = candidate_responses_tags[selected_index]
        selected_response = candidate_responses[selected_index]
        selected_response_embedding= model.encode([selected_response])
        similarity = util.pytorch_cos_sim(input_embedding, selected_response_embedding)

        # Print the selected response
        print("Selected Response:", selected_response)
        # Print the most similar response
        print("Input sentence:", input_sentence)
        print("Most tag:", candidate_responses_tags[most_similar_index])
        print("Most similar response:", candidate_responses[most_similar_index])
        print("similarity:",similarity)
        result = {
            "input_sentence": input_sentence,
            "tag": candidate_responses_tags[most_similar_index],
            "content": selected_response,
            "similarity" : round(similarity.item(),5)
        }
        send_json_response(self, '200', result)

if __name__ == '__main__':
    try:
        port = 8000
        server = HTTPServer(('localhost', port), WebServerHandler)
        server.max_body_length = 10 * 1024 * 1024  # set to 10 MB

        print(f'Server running on port {port}')
        
        server.serve_forever()
    except KeyboardInterrupt:
        print('^C received, shutting down server')
        server.socket.close()


Server running on port 8000
Selected Response: aloha!
Input sentence: hi
Most tag: APPROVE
Most similar response: ok
similarity: tensor([[0.6089]])


127.0.0.1 - - [20/Dec/2023 15:16:02] "POST / HTTP/1.1" 200 -


Selected Response: charming
Input sentence: hi
Most tag: REWARD
Most similar response: very well
similarity: tensor([[0.5001]])


127.0.0.1 - - [20/Dec/2023 15:16:04] "POST / HTTP/1.1" 200 -


Selected Response: roll in 
Input sentence: hi
Most tag: APPROVE
Most similar response: ok
similarity: tensor([[0.5969]])


...

127.0.0.1 - - [20/Dec/2023 16:58:05] "POST / HTTP/1.1" 200 -


Selected Response: 😊
Input sentence: hi
Most tag: BE_GOOD
Most similar response: 😂
similarity: tensor([[0.5746]])


127.0.0.1 - - [20/Dec/2023 16:58:06] "POST / HTTP/1.1" 200 -


Selected Response: hi~
Input sentence: hi
Most tag: CALL
Most similar response: HI
similarity: tensor([[0.7740]])


127.0.0.1 - - [20/Dec/2023 16:58:12] "POST / HTTP/1.1" 200 -
